In [26]:
%pylab inline
import glob, pandas, os, vtk

Populating the interactive namespace from numpy and matplotlib


/usr/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['f', 'normal']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


We want to implemnt this model, on the basis of actual data
$$d =d_0 - \vec p_i \cdot \hat n $$

In [27]:
def get_implicit_model_scalar_value(point, d0, normal):
    return d0 - point.dot(normal)

def get_implicit_model_gradient(point, d0, normal):
    return normal

In [28]:
folder = "data"
ext = ".asc"

files = glob.glob(os.path.join(folder, "*" +  ext))
lines = []
for f in files:
    t = pandas.read_csv(f,header=None, sep=" ")
    lines.append(t)

In [29]:
centers = [np.mean(np.array(l), axis=0) for l in lines]

In [30]:
centers

[array([-25.29972553,  -1.84934804, -13.97334313]),
 array([-25.30842945,  -2.36372157, -13.86027854]),
 array([-25.39080157,  -1.78226469, -13.97001648])]

In [31]:
decentred = [np.array(l) - center for l, center in zip(lines, centers)]

In [32]:
vals, vects = np.linalg.eig(np.cov(np.row_stack(decentred).T))

In [33]:
id = np.argmin(vals)
normal = vects[:,id]

We have the normal of our model, we now need a $d_0$. We place the 0 of our reference system at the very first plane cneter

In [34]:
d0 = -centers[0].dot(normal) # distance of the first centroid from the origin, along the normal direction
print(d0)

12.250683643676703


We now need to define a volume of interest

In [35]:
mesh = "data/mesh.ply"

r  = vtk.vtkPLYReader()
r.SetFileName(mesh)
r.Update()
vtp = r.GetOutput()

In [36]:
from vtk.util.numpy_support import *
allpts = vtk_to_numpy(vtp.GetPoints().GetData())

In [37]:
values = [get_implicit_model_scalar_value(p, d0, normal) for p in allpts]
asarr= numpy_to_vtk(values)
asarr.SetName("values")

In [38]:
vtp.GetPointData().AddArray(asarr)

2

In [39]:
w = vtk.vtkXMLPolyDataWriter()
w.SetFileName("model1.vtp")
w.SetInputData(vtp)
w.Update()


In [44]:
mins = np.min(allpts, axis=0)
maxs = np.max(allpts, axis=0)
print(mins, maxs)

[-26.322306   -2.9138064 -14.601566 ] [-24.038788   -1.0010338 -13.625318 ]


In [43]:
def generateVtkImageFromBoundingBox(bmin, bmax, spacing=0.1):
    vol = vtk.vtkImageData()
    vol.SetSpacing([spacing for i in np.arange(3)])
    nn = np.ceil((bmax - bmin) / spacing)
    nn = nn.astype(int)
    vol.SetExtent(0, nn[0], 0, nn[1], 0, nn[2])
    vol.SetOrigin(mins)
    return vol


def extractPointsFromVtkImage(volume):
    points = []
    for i in np.arange(volume.GetNumberOfPoints()):
        p = np.array(volume.GetPoint(i))
        points.append(p)

    return np.array(points)


In [47]:
vol = generateVtkImageFromBoundingBox(mins, maxs, 0.1)
volpts = extractPointsFromVtkImage(vol)

In [52]:
values = [get_implicit_model_scalar_value(p, d0, normal) for p in volpts]
asarr= numpy_to_vtk(values)
asarr.SetName("values")

In [53]:
vol.GetPointData().AddArray(asarr)

0

In [54]:
w = vtk.vtkXMLImageDataWriter()
w.SetFileName("volume.vti")
w.SetInputData(vol)
w.Write()

1